# Running openQmin (Python interface)

Using *tools/runHelper.py*, you can define command-line parameters through a Python dictionary `runHelper.params`. These, along with any default parameters you didn't change, are converted by `runHelper.get_runcmd()` into a command-line string that calls *build/openQmin.out*.

The dictionary keys of `runHelper.params` are the same as the long forms (appearing after the `--`s) of the command-line flags seen when you run `build/openQmin.out --help`, with the following exceptions:
* `help` itself is not a key in `runHelper.params`
* Parameters `'whole_Lx'`, `'whole_Ly'`, and `'whole_Lz'`, which define the system size **before** subdivision over MPI ranks, override `'Lx'`, `'Ly'`, and `'Lz'` by default. If you want to use `'Lx'`, `'Ly'`, `'Lz'` instead (which give the system size on each rank), you can pass `do_partition=False` to `runHelper.get_runcmd()`.

In the example below, we'll make use of an example boundaryFile that we created in the page on [Boundary conditions (Python interface)](py-boundaries) and the example initialConfigurationFiles that we created in the page on [Initialization (Python interface)](py-initialization). 

Notice that the main openQmin directory path, assigned to `runHelper.directory`, is automatically prepended to the filepaths for imported and exported data. This directory path should be either an absolute path or relative to where you'll be running the command.

In [1]:
from sys import path
path.append("../tools/")  # path to runHelper.py

import runHelper

runHelper.directory = "../" # path to openQmin main directory
runHelper.mpi_num_processes = 3  # set to 1 for non-MPI run

runHelper.params["boundaryFile"] = "ceiling_and_wavy_floor.txt"
runHelper.params["initialConfigurationFile"] = "my_init_file"

# choose a location and filename-prefix for this run's results
runHelper.params["saveFile"] = "data/my_example_run"
runHelper.params["iterations"] = 500  # max number of minimization timesteps

# system size BEFORE subdivision across MPI ranks:
runHelper.params["whole_Lx"] = 50
runHelper.params["whole_Ly"] = 50
runHelper.params["whole_Lz"] = 50

runcmd = runHelper.get_runcmd()  # generate command-line string
print(runcmd)

mpirun -n 3 ../build/openQmin.out --initializationSwitch 0 --GPU -1 --phaseConstantA -0.172 --phaseConstantB -2.12 --phaseConstantC 1.73 --deltaT 0.0005 --fTarget 1e-12 --iterations 500 --randomSeed -1 --L1 4.64 --L2 4.64 --L3 4.64 --L4 4.64 --L6 4.64 --boxL 50 --Lx 16 --Ly 50 --Lz 50 --initialConfigurationFile ../my_init_file --boundaryFile ../ceiling_and_wavy_floor.txt --saveFile ../data/my_example_run --linearSpacedSaving -1 --logSpacedSaving -1 --stride 1 --hFieldX 0 --hFieldY 0 --hFieldZ 0 --hFieldMu0 1 --hFieldChi 1 --hFieldDeltaChi 0.5 --eFieldX 0 --eFieldY 0 --eFieldZ 0 --eFieldEpsilon0 1 --eFieldEpsilon 1 --eFieldDeltaEpsilon 0.5


We can run openQmin with these options by any of the following routes:

* Call `runHelper.run()`, which executes the result of `runHelper.get_runcmd()`
* Copy and paste the string into a terminal
* Use the `runcmd` string in a Python script via `import os; os.system(runcmd)`
* Run as shell command in a Jupyter notebook with `!{runcmd}`

In [2]:
runHelper.run()

--------------------------------------------------------------------------
There are not enough slots available in the system to satisfy the 3
slots that were requested by the application:

  ../build/openQmin.out

Either request fewer slots for your application, or make more slots
available for use.

A "slot" is the Open MPI term for an allocatable unit where we can
launch a process.  The number of slots available are defined by the
environment in which Open MPI processes are run:

  1. Hostfile, via "slots=N" clauses (N defaults to number of
     processor cores if not provided)
  2. The --host command line parameter, via a ":N" suffix on the
     hostname (N defaults to 1 if not provided)
  3. Resource manager (e.g., SLURM, PBS/Torque, LSF, etc.)
  4. If none of a hostfile, the --host command line parameter, or an
     RM is present, Open MPI defaults to the number of processor cores

In all the above cases, if you want Open MPI to default to the number
of hardware threads instead o

Let's take a look at the result using openViewMin.

In [3]:
# Install openViewMin dependencies as needed
!{sys.executable} -m pip install pandas "PyQt5<5.14" pyvistaqt tqdm imageio-ffmpeg >> /dev/null

zsh:1: parse error near `-m'


In [4]:
# replace with your path to openViewMin:
sys.path.append("../../openviewmin/")

import openViewMin
import glob

# collect all files from this run
savedFiles = glob.glob("../data/my_example_run*") 
# generate plot off-screen
nplot = openViewMin.NematicPlot(savedFiles, off_screen=True, window_size=(800, 800))
# rotate plane of director glyphs to y-normal
nplot.update_filter("director_plane", {"normal":[0,1,0]}, update_self_actor=True)
# reduce lighting intensity a bit
nplot.set_lights_intensity(0.6)
# display in notebook
display(nplot.to_pythreejs())
nplot.close()

NameError: name 'sys' is not defined